In [1]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
seed = 0
np.random.seed(seed)

## Fetch the jet tagging dataset from Open ML (run this only once, this can take a while)

In [2]:
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, y = data['data'], data['target']

In [3]:
le = LabelEncoder()
yl = le.fit_transform(y)
yc = to_categorical(yl, len(le.classes_))

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, yc, test_size=0.2, random_state=42)

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
import setGPU

setGPU: Setting GPU to: 3


In [6]:
def History(label, h):
    plt.plot( h.history['loss'], label ='Training Loss: {}'.format(label))
    plt.plot( h.history['val_loss'], label = 'Validation Loss: {}'.format(label))
    plt.plot( h.history['accuracy'], label = 'Training Accuracy: {}'.format(label))
    plt.plot( h.history['val_accuracy'], label = 'Validation Accuracy: {}'.format(label))
    plt.legend()
    plt.show()

In [7]:
def Accuracy(m, X, Y):
    pred = m.predict( X)
    a = accuracy_score(np.argmax(Y, axis=1), np.argmax(pred, axis=1))
    print("Accuracy: {}: {}".format(m.name, a))
    return pred, a

## Construct a simple dense model

We'll use 3 hidden layers with 32, then 64, then 64 neurons. Each layer will use `relu` activation.
Add an output layer with 5 neurons (one for each class), then finish with Softmax activation.

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense, Activation, BatchNormalization, Conv1D, Dropout, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

In [9]:
i = Input(shape=(16,))
d = Dense(32, name='fc1', activation='relu')(i)
d = Dense(64, name='fc2', activation='relu')(d)
d = Dense(64, name='fc3', activation='relu')(d)
o = Dense(len(le.classes_), activation='softmax', name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001))(d)
model = Model(inputs=i, outputs=o, name='model_DNN')

In [10]:
model.summary()

Model: "model_DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 32)                544       
_________________________________________________________________
fc2 (Dense)                  (None, 64)                2112      
_________________________________________________________________
fc3 (Dense)                  (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 5)                 325       
Total params: 7,141
Trainable params: 7,141
Non-trainable params: 0
_________________________________________________________________


## Train the model
We'll use Adam optimizer with categorical crossentropy loss.
The model isn't very complex, so this should just take a few minutes even on the CPU.

In [11]:
from sklearn.model_selection import StratifiedKFold

In [12]:
n_folds=10
folding = StratifiedKFold(n_splits=n_folds)

In [13]:
histories=[]
accuracies=[]
models=[]
for i_fold, (train_index, test_index) in enumerate(folding.split(X, y)):
    X_fold_train, X_fold_test = X[train_index], X[test_index]
    y_fold_train, y_fold_test = yc[train_index], yc[test_index]
    models.append( Model(inputs=i, outputs=o, name='model_DNN_{}'.format(i_fold)) )
    models[-1].compile(optimizer=Adam(lr=0.0001), loss=['categorical_crossentropy'], metrics=['accuracy'])
    histories.append(models[-1].fit( X_fold_train, y_fold_train, batch_size=1024,
                    epochs=2, validation_split=0.25, shuffle=True) )
    _, a = Accuracy(models[-1], X_fold_test, y_fold_test)
    accuracies.append( a )

Epoch 1/2
548/548 [==============================] - 9s 16ms/step - loss: 1.5251 - accuracy: 0.3539 - val_loss: 1.3430 - val_accuracy: 0.4088
Epoch 2/2
548/548 [==============================] - 7s 12ms/step - loss: 1.3075 - accuracy: 0.4407 - val_loss: 1.2641 - val_accuracy: 0.4789
Accuracy: model_DNN_0: 0.4762409638554217
Epoch 1/2
548/548 [==============================] - 7s 13ms/step - loss: 1.2115 - accuracy: 0.5403 - val_loss: 1.1558 - val_accuracy: 0.5839
Epoch 2/2
548/548 [==============================] - 7s 13ms/step - loss: 1.1147 - accuracy: 0.5941 - val_loss: 1.0791 - val_accuracy: 0.6055
Accuracy: model_DNN_1: 0.6050481927710843
Epoch 1/2
548/548 [==============================] - 7s 12ms/step - loss: 1.0549 - accuracy: 0.6134 - val_loss: 1.0352 - val_accuracy: 0.6242
Epoch 2/2
548/548 [==============================] - 6s 10ms/step - loss: 1.0213 - accuracy: 0.6310 - val_loss: 1.0091 - val_accuracy: 0.6397
Accuracy: model_DNN_2: 0.6377710843373494
Epoch 1/2
548/548 [===

In [14]:
print("The model accuracy is {:0.3f} +/- {:0.3f} over {} folds".format(np.mean(accuracies),np.std(accuracies), n_folds ))

The model accuracy is 0.646 +/- 0.062 over 10 folds


# Exercise
Implement this mechanism to the starter notebook

# Challenge
Can the number of folds be reduced as the RMS on performance is reducing?